In [1]:
import numpy as np
import pandas as pd
import math
import base64

### Chicago Crash Data Cleaning || Feature Engineering || Formatting

In [2]:
#-------------- Read Chicago Crash Data

crash_df = pd.read_csv("chicago_data/traffic_crashes_2022.csv")

In [3]:
#-------------- Check Dataset Shape

crash_df.shape

(576357, 49)

In [4]:
#-------------- Check Columns and Dataset

#print(crash_df.columns)
crash_df.head(2)

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,JC343143,NaN,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)


In [5]:
#-------------- Extract All Relevant Fields

relevent_columns = ["CRASH_RECORD_ID","CRASH_DATE","POSTED_SPEED_LIMIT",
                    "WEATHER_CONDITION","ROADWAY_SURFACE_COND","ROAD_DEFECT",
                    "CRASH_TYPE","NUM_UNITS","INJURIES_TOTAL","INJURIES_FATAL",
                    "INJURIES_INCAPACITATING","CRASH_HOUR","CRASH_DAY_OF_WEEK",
                    "CRASH_MONTH","LATITUDE","LONGITUDE","LOCATION"]
crash_df_v2 = crash_df.copy()
crash_df_v2 = crash_df_v2[relevent_columns]
crash_df_v2.head(2)

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,07/10/2019 05:56:00 PM,35,CLEAR,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,2.0,0.0,0.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,06/30/2017 04:00:00 PM,35,CLEAR,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,2.0,0.0,0.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)


In [6]:
"""
No need to rename columns as they are aleady named properly
Look into values in columns to evalue restructure (re-categorize/rename)
Assess the need to convert data types
"""

'\nNo need to rename columns as they are aleady named properly\nLook into values in columns to evalue restructure (re-categorize/rename)\nAssess the need to convert data types\n'

In [7]:
#-------------- Convert CRASH_DATE to pandas datetime format
#-------------- Add feature fields year and day of week (Monday=0, Sunday=6)

crash_df_v2["CRASH_DATE"] = pd.to_datetime(crash_df_v2["CRASH_DATE"])
crash_df_v2["YEAR"]       = crash_df_v2["CRASH_DATE"].dt.year
crash_df_v2["WEEK_DAY_NAME"]   = crash_df_v2["CRASH_DATE"].dt.day_name()

#-------------- Original CRASH_DAY_OF_WEEK field does not align with pandas day of week so drop the field

crash_df_v2 = crash_df_v2.drop(columns=["CRASH_DAY_OF_WEEK"])

crash_df_v2.tail(2)

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,YEAR,WEEK_DAY_NAME
576355,d51aae396db49981c7ee26ceb54dfcab3c4b06d0cc5d7d...,2021-07-12 10:00:00,30,CLEAR,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,2.0,1.0,0.0,0.0,10,7,41.752918,-87.621573,POINT (-87.621572939578 41.752917594949),2021,Monday
576356,c83fa5092963bda54b781bf8cfdd840b5106004afbdd34...,2021-07-13 22:30:00,30,CLEAR,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,3.0,0.0,0.0,0.0,22,7,41.804003,-87.603354,POINT (-87.603354308388 41.804002931352),2021,Tuesday


In [8]:
#-------------- Check data size over years

"""

Since proper data collection started around Oct 2017
accoriding to our data source, it is best to narrow analysis from 2018 - present

"""
#crash_df_v2["YEAR"].value_counts()
crash_df_v2.groupby("YEAR")["YEAR"].count().sort_values(ascending=False)

YEAR
2018    118947
2019    117758
2021    108638
2020     92104
2017     83785
2016     44297
2015      9828
2022       993
2014         6
2013         1
Name: YEAR, dtype: int64

In [9]:
crash_df_v3 = crash_df_v2.copy()

crash_df_v3 = crash_df_v3[(crash_df_v3["YEAR"] >= 2018)&(crash_df_v3["YEAR"] < 2022)]
crash_df_v3["YEAR"].value_counts()

2018    118947
2019    117758
2021    108638
2020     92104
Name: YEAR, dtype: int64

In [10]:
#-------------- Check For Null/Missing Values
print(crash_df_v3.isnull().sum())
crash_df_v3.shape

CRASH_RECORD_ID               0
CRASH_DATE                    0
POSTED_SPEED_LIMIT            0
WEATHER_CONDITION             0
ROADWAY_SURFACE_COND          0
ROAD_DEFECT                   0
CRASH_TYPE                    0
NUM_UNITS                     0
INJURIES_TOTAL              965
INJURIES_FATAL              965
INJURIES_INCAPACITATING     965
CRASH_HOUR                    0
CRASH_MONTH                   0
LATITUDE                   2819
LONGITUDE                  2819
LOCATION                   2819
YEAR                          0
WEEK_DAY_NAME                 0
dtype: int64


(437447, 18)

In [11]:
"""

Only 7 Records where Location and total injuries reported both holding null values

INJURIES_TOTAL holds 943 total missing values as reported above
LOCATION holds 2,758 missing values as reported above

428,394 records where both INJURIES_TOTAL & LOCATION without any missing values

99.14% of records of crash_df_v3 without null values in both Location (lat,log) and injuries fields
"""

df1 = crash_df_v3.isna()
df2 = pd.crosstab(df1.LOCATION, df1.INJURIES_TOTAL)
print (df2)

print("")
print(crash_df_v3.shape[0])

print((df2[0][0]/(crash_df_v3.shape[0]))*100)


INJURIES_TOTAL   False  True 
LOCATION                     
False           433670    958
True              2812      7

437447
99.13658111725535


In [12]:
#-------------- There are 2,765 records with missing location (lat/log) values
#-------------- There are 950 records with missing total/fatal/incapacitatin injuries values
#-------------- Check percentage of missing location & injuries value relative to all available date

location_null = (1 - (crash_df_v3["LOCATION"].value_counts().sum()/crash_df_v3.shape[0]))*100
injuries_null = (1 - (crash_df_v3["INJURIES_TOTAL"].value_counts().sum()/crash_df_v3.shape[0]))*100

missing_total_pct = location_null + injuries_null

print(missing_total_pct)


#-------------- Less than 1 percent of data have missing value under crash_df_v3 
#-------------- It is statistically insignificant to drop those missing record 

0.865019076596707


In [13]:
#-------------- Drop data with missing location records

crash_df_v4 = crash_df_v3.copy()

crash_df_v4 = crash_df_v4.dropna(subset=["LOCATION","INJURIES_TOTAL"])

print(crash_df_v4.isnull().sum())
print("")
print(crash_df_v3.shape)
print(crash_df_v4.shape)

CRASH_RECORD_ID            0
CRASH_DATE                 0
POSTED_SPEED_LIMIT         0
WEATHER_CONDITION          0
ROADWAY_SURFACE_COND       0
ROAD_DEFECT                0
CRASH_TYPE                 0
NUM_UNITS                  0
INJURIES_TOTAL             0
INJURIES_FATAL             0
INJURIES_INCAPACITATING    0
CRASH_HOUR                 0
CRASH_MONTH                0
LATITUDE                   0
LONGITUDE                  0
LOCATION                   0
YEAR                       0
WEEK_DAY_NAME              0
dtype: int64

(437447, 18)
(433670, 18)


In [14]:
#-------------- Check one field at a time to evaluate need to restructure field

crash_df_v4.head(2)

,CRASH_RECORD_ID,CRASH_DATE,POSTED_SPEED_LIMIT,WEATHER_CONDITION,ROADWAY_SURFACE_COND,ROAD_DEFECT,CRASH_TYPE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,CRASH_HOUR,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,YEAR,WEEK_DAY_NAME
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,2019-07-10 17:56:00,35,CLEAR,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,2.0,0.0,0.0,0.0,17,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993),2019,Wednesday
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,2020-07-10 10:25:00,30,CLEAR,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,3.0,0.0,0.0,0.0,10,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008),2020,Friday


In [15]:
"""
For meaningful EDA on weather condition impact on chicago crash outcomes
it is best to recategorize the field based on value relations

"""
crash_df_v4["WEATHER_CONDITION"].value_counts()

CLEAR                       343583
RAIN                         37099
UNKNOWN                      19395
SNOW                         16745
CLOUDY/OVERCAST              13223
OTHER                         1378
FREEZING RAIN/DRIZZLE          713
FOG/SMOKE/HAZE                 709
SLEET/HAIL                     590
BLOWING SNOW                   164
SEVERE CROSS WIND GATE          69
BLOWING SAND, SOIL, DIRT         2
Name: WEATHER_CONDITION, dtype: int64

In [16]:
weather_dict = {
    "BLOWING SAND, SOIL, DIRT" : "RAIN_SNOW_CLOUDY_etc",
    "BLOWING SNOW"             : "RAIN_SNOW_CLOUDY_etc",
    "CLEAR"                    : "CLEAR",
    "CLOUDY/OVERCAST"          : "RAIN_SNOW_CLOUDY_etc",
    "FOG/SMOKE/HAZE"           : "RAIN_SNOW_CLOUDY_etc",
    "FREEZING RAIN/DRIZZLE"    : "RAIN_SNOW_CLOUDY_etc",
    "OTHER"                    : "UKNOWN_OTHER",
    "RAIN"                     : "RAIN_SNOW_CLOUDY_etc",
    "SEVERE CROSS WIND GATE"   : "RAIN_SNOW_CLOUDY_etc",
    "SLEET/HAIL"               : "RAIN_SNOW_CLOUDY_etc",
    "SNOW"                     : "RAIN_SNOW_CLOUDY_etc",
    "UNKNOWN"                  : "UKNOWN_OTHER"
    
}

crash_df_v4["WEATHER_CONDITION"] = crash_df_v4["WEATHER_CONDITION"].replace(weather_dict)
crash_df_v4["WEATHER_CONDITION"].value_counts()

CLEAR                   343583
RAIN_SNOW_CLOUDY_etc     69314
UKNOWN_OTHER             20773
Name: WEATHER_CONDITION, dtype: int64

In [17]:
"""
For meaningful EDA on road surface condition impact on chicago crash outcomes
it is best to recategorize the field based on value relations

"""
crash_df_v4["ROADWAY_SURFACE_COND"].value_counts()

DRY                324291
WET                 58362
UNKNOWN             29927
SNOW OR SLUSH       16753
ICE                  3070
OTHER                1090
SAND, MUD, DIRT       177
Name: ROADWAY_SURFACE_COND, dtype: int64

In [18]:
road_dict = {
    "DRY"             : "DRY",
    "WET"             : "WET_SNOW_ICE_etc",
    "UNKNOWN"         : "UNKNOWN_OTHER",
    "SNOW OR SLUSH"   : "WET_SNOW_ICE_etc",
    "ICE"             : "WET_SNOW_ICE_etc",
    "OTHER"           : "UNKNOWN_OTHER",
    "SAND, MUD, DIRT" : "WET_SNOW_ICE_etc"
    
}

crash_df_v4["ROADWAY_SURFACE_COND"] = crash_df_v4["ROADWAY_SURFACE_COND"].replace(road_dict)
crash_df_v4["ROADWAY_SURFACE_COND"].value_counts()

DRY                 324291
WET_SNOW_ICE_etc     78362
UNKNOWN_OTHER        31017
Name: ROADWAY_SURFACE_COND, dtype: int64

In [19]:
"""
For meaningful EDA on road defect impact on chicago crash outcomes
it is best to recategorize the field based on value relations

"""
crash_df_v4["ROAD_DEFECT"].value_counts()

NO DEFECTS           358301
UNKNOWN               66467
RUT, HOLES             3550
OTHER                  2437
WORN SURFACE           1678
SHOULDER DEFECT         886
DEBRIS ON ROADWAY       351
Name: ROAD_DEFECT, dtype: int64

In [20]:
road_defect_dict = {
    "NO DEFECTS"        : "NO_DEFECTS",
    "UNKNOWN"           : "UNKNOWN_OTHER",
    "RUT, HOLES"        : "HOLES_DEBRIS_WORN_etc",
    "OTHER"             : "UNKNOWN_OTHER",
    "WORN SURFACE"      : "HOLES_DEBRIS_WORN_etc",
    "SHOULDER DEFECT"   : "HOLES_DEBRIS_WORN_etc",
    "DEBRIS ON ROADWAY" : "HOLES_DEBRIS_WORN_etc"
    
}

crash_df_v4["ROAD_DEFECT"] = crash_df_v4["ROAD_DEFECT"].replace(road_defect_dict)
crash_df_v4["ROAD_DEFECT"].value_counts()

NO_DEFECTS               358301
UNKNOWN_OTHER             68904
HOLES_DEBRIS_WORN_etc      6465
Name: ROAD_DEFECT, dtype: int64

In [21]:
"""
Check how may many injuries per each crash...

"""

crash_df_v4["INJURIES_TOTAL"].value_counts()

0.0     370667
1.0      47520
2.0      10474
3.0       3190
4.0       1143
5.0        401
6.0        159
7.0         62
8.0         18
9.0         14
10.0         7
15.0         4
21.0         3
11.0         3
12.0         2
13.0         1
16.0         1
19.0         1
Name: INJURIES_TOTAL, dtype: int64

In [22]:
"""
It appears that 99.94% percent of the crashes involved sustained 5 injuries or less,
it is best to exclude crashes with more than 5 injuries since it will be statistically insignificant 
in addition we'll later join with crash people data which will eliminate unneccessary join records down the line

"""
(1 - (crash_df_v4[crash_df_v4["INJURIES_TOTAL"]>5]["INJURIES_TOTAL"].value_counts().sum()/crash_df_v4.shape[0]))*100

99.93658772799594

In [23]:
crash_df_v4["INJURIES_INCAPACITATING"].value_counts()

0.0    425563
1.0      7124
2.0       737
3.0       172
4.0        56
5.0        15
6.0         2
7.0         1
Name: INJURIES_INCAPACITATING, dtype: int64

In [24]:
#-------------- Similar Outcomes holds for incapacitating injuries majority being 5 or less per crash (99.99%)
(1 - (crash_df_v4[crash_df_v4["INJURIES_INCAPACITATING"]>5]["INJURIES_INCAPACITATING"].value_counts().sum()/crash_df_v4.shape[0]))*100

99.99930822975995

In [25]:
#-------------- Drop total injuries and incapacitating injuries more than 5 per crash

crash_df_v5 = crash_df_v4.copy()

crash_df_v5 = crash_df_v5[crash_df_v5["INJURIES_TOTAL"]<=5]
crash_df_v5 = crash_df_v5[crash_df_v5["INJURIES_INCAPACITATING"]<=5]

In [26]:
#-------------- Check data types and evaluate need to change data type for a given field


In [27]:
crash_df_v5.dtypes

print("")
print(crash_df_v4.shape)
print(crash_df_v5.shape)


(433670, 18)
(433395, 18)


In [28]:
#-------------- Convert some of the columns data type to int for improved EDA later on
cols = ["INJURIES_TOTAL", "INJURIES_FATAL", "INJURIES_INCAPACITATING"]

for x in cols:
    crash_df_v5[x] = crash_df_v5[x].astype("int")


In [29]:
crash_df_v5.dtypes

CRASH_RECORD_ID                    object
CRASH_DATE                 datetime64[ns]
POSTED_SPEED_LIMIT                  int64
WEATHER_CONDITION                  object
ROADWAY_SURFACE_COND               object
ROAD_DEFECT                        object
CRASH_TYPE                         object
NUM_UNITS                         float64
INJURIES_TOTAL                      int32
INJURIES_FATAL                      int32
INJURIES_INCAPACITATING             int32
CRASH_HOUR                          int64
CRASH_MONTH                         int64
LATITUDE                          float64
LONGITUDE                         float64
LOCATION                           object
YEAR                                int64
WEEK_DAY_NAME                      object
dtype: object

In [30]:
#-------------- crash_df_v5[crash_df_v5.duplicated(["CRASH_RECORD_ID"], keep=False)]

print(crash_df_v5[crash_df_v5["CRASH_RECORD_ID"].duplicated() == True].shape[0])

print(crash_df_v5[crash_df_v5["CRASH_RECORD_ID"].duplicated() == True].size)


#-------------- CRASH_RECORD_ID holds unique values and no duplicates thus can serve as primary
#-------------- We'll use CRASH_RECORD_ID to join with crash people table later on

0
0


In [31]:
#-------------- Store final result as a csv file
crash_df_v5.to_csv("chicago_data/chicago_clean_crash_2022.csv", encoding="utf-8")

### Chicago Crash_People Data Cleaning || Feature Engineering || Formatting

In [32]:
#-------------- Read Chicago Crash People Data

crash_people_df = pd.read_csv("chicago_data/traffic_crashes_people_2022.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (12,13,20,21,22,23,24,25,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
#-------------- Check Dataset Shape

crash_people_df.shape

(1273696, 30)

In [34]:
#-------------- Check Columns and Dataset

#print(crash_people_df.columns)
crash_people_df.head(2)

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [35]:
#-------------- Extract All Relevant Fields

sub_columns = ["PERSON_ID","PERSON_TYPE","CRASH_RECORD_ID",
                "CRASH_DATE","SEX","AGE","INJURY_CLASSIFICATION",
                "DRIVERS_LICENSE_STATE","CITY","STATE","ZIPCODE"]
crash_people_df_v2 = crash_people_df.copy()
crash_people_df_v2 = crash_people_df_v2[sub_columns]
crash_people_df_v2.head(2)

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,CRASH_DATE,SEX,AGE,INJURY_CLASSIFICATION,DRIVERS_LICENSE_STATE,CITY,STATE,ZIPCODE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,M,25.0,NO INDICATION OF INJURY,IL,CHICAGO,IL,60651
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,04/13/2020 10:50:00 PM,M,37.0,NO INDICATION OF INJURY,IL,CHICAGO,IL,60620


In [36]:
"""
No need to rename columns as they are aleady named properly
Look into values in columns to evalue restructure (re-categorize/rename)
Assess the need to convert data types

"""

'\nNo need to rename columns as they are aleady named properly\nLook into values in columns to evalue restructure (re-categorize/rename)\nAssess the need to convert data types\n\n'

In [37]:
#-------------- Convert CRASH_DATE to pandas datetime format
#-------------- No need to retrive day of week since it's present on crash data

crash_people_df_v2["CRASH_DATE"] = pd.to_datetime(crash_people_df_v2["CRASH_DATE"])
crash_people_df_v2["YEAR"]       = crash_people_df_v2["CRASH_DATE"].dt.year

crash_people_df_v2.tail(2)

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,CRASH_DATE,SEX,AGE,INJURY_CLASSIFICATION,DRIVERS_LICENSE_STATE,CITY,STATE,ZIPCODE,YEAR
1273694,P280239,PASSENGER,f4662e1e6bad3a2a197f6990d55ceb30a7d464493e350c...,2021-12-20 19:05:00,M,NaN,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,2021
1273695,P280240,PASSENGER,f4662e1e6bad3a2a197f6990d55ceb30a7d464493e350c...,2021-12-20 19:05:00,F,NaN,NO INDICATION OF INJURY,NaN,NaN,NaN,NaN,2021


In [38]:
#-------------- We are only interested from 2018-Present, because of data availability

crash_people_df_v3 = crash_people_df_v2.copy()

crash_people_df_v3 = crash_people_df_v3[(crash_people_df_v3["YEAR"] >= 2018)&(crash_people_df_v3["YEAR"] < 2022)]
crash_people_df_v3["YEAR"].value_counts()

2018    265702
2019    263985
2021    237530
2020    202123
Name: YEAR, dtype: int64

In [39]:
#-------------- Check For Null/Missing Values
print(crash_people_df_v3.isnull().sum())
crash_people_df_v3.shape

PERSON_ID                     0
PERSON_TYPE                   0
CRASH_RECORD_ID               0
CRASH_DATE                    0
SEX                       15325
AGE                      276179
INJURY_CLASSIFICATION       373
DRIVERS_LICENSE_STATE    400701
CITY                     259707
STATE                    248363
ZIPCODE                  321454
YEAR                          0
dtype: int64


(969340, 12)

In [40]:
"""
There are statistically significant number of missing records
under DRIVERS_LICENSE_STATE, CITY, STATE, ZIPCODE
Let's drop the fields unless a team member finds a good reason to retain these fields
"""
drop_fields = ["DRIVERS_LICENSE_STATE","CITY","STATE","ZIPCODE"]
crash_people_df_v3.drop(drop_fields, axis=1, inplace=True)
print(crash_people_df_v3.isnull().sum())


PERSON_ID                     0
PERSON_TYPE                   0
CRASH_RECORD_ID               0
CRASH_DATE                    0
SEX                       15325
AGE                      276179
INJURY_CLASSIFICATION       373
YEAR                          0
dtype: int64


In [41]:
#-------------- Let's look deeper into SEX and AGE fields and handle missing values
#-------------- Replace missing values with Unknown/notReported under SEX column

crash_people_df_v4 = crash_people_df_v3.copy()
crash_people_df_v4["SEX"] = crash_people_df_v4["SEX"].fillna("NOT_RECORDED")
crash_people_df_v4["SEX"].value_counts()

M               504338
F               363682
X                85995
NOT_RECORDED     15325
Name: SEX, dtype: int64

In [42]:
sex_dict = {
    "M"            : "MALE",
    "F"            : "FEMALE",
    "X"            : "UNKNOWN_OTHER",
    "NOT_RECORDED" : "UNKNOWN_OTHER"
    
}

crash_people_df_v4["SEX"] = crash_people_df_v4["SEX"].replace(sex_dict)
crash_people_df_v4["SEX"].value_counts()

MALE             504338
FEMALE           363682
UNKNOWN_OTHER    101320
Name: SEX, dtype: int64

In [43]:
#-------------- Replace missing values under AGE field with median value

age_median = crash_people_df_v4["AGE"].median()
age_median

crash_people_df_v4["AGE"] = crash_people_df_v4["AGE"].fillna(age_median)

In [44]:
print(crash_people_df_v4.isnull().sum())
print("")
print(crash_people_df_v3.shape)
print(crash_people_df_v4.shape)

#-------------- Successfully handled missing value and maintained data size from version3

PERSON_ID                  0
PERSON_TYPE                0
CRASH_RECORD_ID            0
CRASH_DATE                 0
SEX                        0
AGE                        0
INJURY_CLASSIFICATION    373
YEAR                       0
dtype: int64

(969340, 8)
(969340, 8)


In [45]:
#-------------- Check data types and evaluate need to change data type for a given field

crash_people_df_v4.dtypes

PERSON_ID                        object
PERSON_TYPE                      object
CRASH_RECORD_ID                  object
CRASH_DATE               datetime64[ns]
SEX                              object
AGE                             float64
INJURY_CLASSIFICATION            object
YEAR                              int64
dtype: object

In [46]:
#-------------- Convert Age datatype from float to int
crash_people_df_v4["AGE"] = crash_people_df_v4["AGE"].astype("int")

In [47]:

print(crash_people_df_v4[crash_people_df_v4["PERSON_ID"].duplicated() == True].shape[0])

print(crash_people_df_v4[crash_people_df_v4["PERSON_ID"].duplicated() == True].size)


#-------------- PERSON_ID holds unique values and no duplicates 


0
0


In [48]:
#cols = list(crash_people_df_v4.columns.values)
cols = ['PERSON_ID','PERSON_TYPE','CRASH_RECORD_ID','CRASH_DATE',"INJURY_CLASSIFICATION",'YEAR','SEX','AGE']

crash_people_df_v5 = crash_people_df_v4.copy()
crash_people_df_v5 = crash_people_df_v5.reindex(columns=cols)
crash_people_df_v5.head(2)

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,CRASH_DATE,INJURY_CLASSIFICATION,YEAR,SEX,AGE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,2019-09-28 03:30:00,NO INDICATION OF INJURY,2019,MALE,25
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,2020-04-13 22:50:00,NO INDICATION OF INJURY,2020,MALE,37


In [49]:
print("")
print(crash_people_df_v3.shape)
print(crash_people_df_v4.shape)
print(crash_people_df_v5.shape)


(969340, 8)
(969340, 8)
(969340, 8)


In [50]:
#-------------- Store final result as a csv file
crash_people_df_v5.to_csv("chicago_data/chicago_clean_crash_people_2022.csv", encoding="utf-8")

### Chicago Crash_Vehicle Data Cleaning || Feature Engineering || Formatting

In [51]:
#-------------- Read Chicago Crash Vehicles Data

vehicle_df = pd.read_csv("chicago_data/traffic_crashes_vehicles_2022.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
vehicle_df.shape

(1186447, 72)

In [53]:
#-------------- Check Columns and Dataset


vehicle_df.head(2)

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,JD124535,01/22/2020 06:25:00 AM,1,DRIVER,NaN,796949.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
#-------------- Extract All Relevant Fields

sub_columns = ["CRASH_RECORD_ID","UNIT_TYPE","VEHICLE_YEAR"]
vehicle_df_v2 = vehicle_df.copy()
vehicle_df_v2 = vehicle_df_v2[sub_columns]
vehicle_df_v2.head(2)

,CRASH_RECORD_ID,UNIT_TYPE,VEHICLE_YEAR
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,DRIVER,2017.0
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,DRIVER,2016.0


In [55]:
"""
Earliers car manufacture year in the dataset is 1900
"""
vehicle_df_v2.groupby(["VEHICLE_YEAR"]).size()


VEHICLE_YEAR
1900.0    250
1901.0     15
1905.0      1
1909.0      1
1911.0      1
         ... 
8001.0      1
8609.0      1
9418.0      1
9900.0      2
9999.0    320
Length: 196, dtype: int64

In [56]:
#------------------- Vehicle make year need to be on or after 1990 and on or before 2021 
vehicle_df_v3 = vehicle_df_v2[(vehicle_df_v2["VEHICLE_YEAR"]>=1990.0) & 
                                            (vehicle_df_v2["VEHICLE_YEAR"]<=2021.0)]

In [57]:
vehicle_df_v3["VEHICLE_YEAR"] = vehicle_df_v3["VEHICLE_YEAR"].astype("int")

<ipython-input-57-1bb9ed6fe2d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_df_v3["VEHICLE_YEAR"] = vehicle_df_v3["VEHICLE_YEAR"].astype("int")


In [58]:
vehicle_df_v3.head(2)

,CRASH_RECORD_ID,UNIT_TYPE,VEHICLE_YEAR
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,DRIVER,2017
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,DRIVER,2016


In [59]:
"""

vehicle age classification based on vehicle make year

2021 is max treshold year when crashes occurred & reported

Logic for For Loop is;

if vehicle production year is greater than 2011 (2021 - 10) lable as 'LessThan10Yrs'
else if vehicle production year is less than or equals to 2011 & greater than or equals to 2006 (2021-15) label as '10-15Yrs'
else if vehicle production year is less than or equals to 2005 (2021 - 16) & greater than or equals to 2001 (2021 - 20) label as '16-20Yrs'
else label 'GreaterThan20Yrs'
"""

vehicle_age = []

for x in vehicle_df_v3["VEHICLE_YEAR"]:
    
    if x > 2011:
        vehicle_age.append("LessThan10Yrs")
    elif (x <= 2011) & (x >= 2006):
        vehicle_age.append("10-15Yrs")
    elif (x <= 2005) & (x >= 2001):
        vehicle_age.append("16-20Yrs")
    else:
        vehicle_age.append("GreaterThan20Yrs")
        
vehicle_df_v3["VEHICLE_AGE"] = vehicle_age
vehicle_df_v3.head(2)


<ipython-input-59-ad51c9ebcd7e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_df_v3["VEHICLE_AGE"] = vehicle_age


,CRASH_RECORD_ID,UNIT_TYPE,VEHICLE_YEAR,VEHICLE_AGE
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,DRIVER,2017,LessThan10Yrs
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,DRIVER,2016,LessThan10Yrs


In [60]:
#--------------------------- For vehicles in crashes, only interested with controlled(driver's active) vehicles
vehicle_df_v3["UNIT_TYPE"].value_counts()

DRIVER                 811696
PARKED                 144677
DRIVERLESS               9783
DISABLED VEHICLE          115
NON-CONTACT VEHICLE        60
Name: UNIT_TYPE, dtype: int64

In [61]:
vehicle_df_v4 = vehicle_df_v3.copy()
vehicle_df_v4 = vehicle_df_v4[vehicle_df_v4["UNIT_TYPE"]=="DRIVER"]

In [62]:
vehicle_df_v4["UNIT_TYPE"].value_counts()

DRIVER    811696
Name: UNIT_TYPE, dtype: int64

In [63]:
#-------------- Store final result as a csv file
vehicle_df_v4.to_csv("chicago_data/chicago_clean_crash_vehicles_2022.csv", encoding="utf-8")

### Chicago Transit Authority Data Cleaning || Feature Engineering || Formatting

Ridership Readme 12-Aug-2011 Chicago Transit Authority

* About CTA ridership numbers * Ridership statistics are provided on a system-wide and bus route/station-level basis. Ridership is primarily counted as boardings, that is, customers boarding a transit vehicle (bus or rail). On the rail system, there is a distinction between station entries and total rides, or boardings. Datasets indicate such in their file name and description.

* How people are counted on the 'L' * On the rail system, a customer is counted as an "entry" each time he or she passes through a turnstile to enter a station. Customers are not counted as "entries" when they make a "cross-platform" transfer from one rail line to another, since they don't pass through a turnstile. Where the number given for rail is in "boardings," what's presented is a statistically valid estimate of the actual number of boardings onto the rail system.

* How people are counted on buses * Boardings are recorded using the bus farebox and farecard reader. In the uncommon situation when there is an operating error with the farebox and the onboard systems cannot determine on which route a given trip's boardings should be allocated, these boardings are tallied as Route 0 in some reports. Route 1001 are shuttle buses used for construction or other unforeseen events.

* "Daytype" * Daytype fields in the data are coded as "W" for Weekday, "A" for Saturday and "U" for Sunday/Holidays. Note that New Year's Day, Memorial Day, Independence Day, Labor Day, Thanksgiving, and Christmas Day are considered as "Sundays" for the purposes of ridership reporting. All other holidays are reported as the type of day they fall on.`m

In [64]:
#-------------- Read Chicago Transit Authority Data

cta_df = pd.read_csv("chicago_data/CTA_ridership_daily_boarding_totals.csv")
cta_df.head(2)

,service_date,day_type,bus,rail_boardings,total_rides
0,01/01/2001,U,297192,126455,423647
1,01/02/2001,W,780827,501952,1282779


In [65]:
#-------------- Check for null/missing values

cta_df.isnull().sum()

#-------------- No missing values in all data filds

service_date      0
day_type          0
bus               0
rail_boardings    0
total_rides       0
dtype: int64

In [66]:
#-------------- Rename columns
cta_df.columns

Index(['service_date', 'day_type', 'bus', 'rail_boardings', 'total_rides'], dtype='object')

In [67]:
cols_rename = { 
                "service_date"   : "SERVICE_DATE",
                "day_type"       : "WEEK_DAY1", 
                'bus'            : "BUS_RIDES", 
                "rail_boardings" : "RAIL_BOARDINGS",
                "total_rides"    : "TOTAL_RIDES"}

cta_df = cta_df.rename(columns=cols_rename)

In [68]:
#-------------- Convert SERVICE_DATE to pandas datetime

cta_df["SERVICE_DATE"] = pd.to_datetime(cta_df["SERVICE_DATE"])

In [69]:
#-------------- Add feature fields spriging from SERVICE_DATE field
cta_df["YEAR"]        = cta_df["SERVICE_DATE"].dt.year
cta_df["MONTH"]       = cta_df["SERVICE_DATE"].dt.month
cta_df["WEEK_DAY2"]   = cta_df["SERVICE_DATE"].dt.weekday
cta_df["WEEK_DAY"]    = cta_df["SERVICE_DATE"].dt.day_name()

In [70]:
cta_df.head(5)

,SERVICE_DATE,WEEK_DAY1,BUS_RIDES,RAIL_BOARDINGS,TOTAL_RIDES,YEAR,MONTH,WEEK_DAY2,WEEK_DAY
0,2001-01-01,U,297192,126455,423647,2001,1,0,Monday
1,2001-01-02,W,780827,501952,1282779,2001,1,1,Tuesday
2,2001-01-03,W,824923,536432,1361355,2001,1,2,Wednesday
3,2001-01-04,W,870021,550011,1420032,2001,1,3,Thursday
4,2001-01-05,W,890426,557917,1448343,2001,1,4,Friday


In [71]:
#-------------- Rename WEEK_DAY1 (original data week day) encoding and alter from string/object to category datatype
days_rename         = {"A" : "Saturday", "U" : "Sunday", "W" : "Weekday"}
cta_df["WEEK_DAY1"] = cta_df["WEEK_DAY1"].replace(days_rename)
cta_df.head(2)

,SERVICE_DATE,WEEK_DAY1,BUS_RIDES,RAIL_BOARDINGS,TOTAL_RIDES,YEAR,MONTH,WEEK_DAY2,WEEK_DAY
0,2001-01-01,Sunday,297192,126455,423647,2001,1,0,Monday
1,2001-01-02,Weekday,780827,501952,1282779,2001,1,1,Tuesday


In [72]:
"""
On the original dataset day of week translated to  "W" for Weekday, "A" for 
Saturday and "U" for Sunday/Holidays, thus on the feature field, WEEK_DAY_NAME,
and the original WEEK_DAY1 field don't much on Sunday then it is a holiday, 
as it was flaged Sunday on the original data field on a day that was not Sunday..
"""

'\nOn the original dataset day of week translated to  "W" for Weekday, "A" for \nSaturday and "U" for Sunday/Holidays, thus on the feature field, WEEK_DAY_NAME,\nand the original WEEK_DAY1 field don\'t much on Sunday then it is a holiday, \nas it was flaged Sunday on the original data field on a day that was not Sunday..\n'

In [73]:
#-------------- Add a Holiday Field based on exisitng field values (Yes:Holiday, No:Not Holiday)

cta_df["HOLIDAY"] = np.where((cta_df["WEEK_DAY1"]=="Sunday")&(cta_df["WEEK_DAY"]!="Sunday"),"Yes","No")



In [74]:
#-------------- Drop WEEK_DAY1 & WEEK_DAY as they are redundant
cta_df = cta_df.drop(columns=["WEEK_DAY1","WEEK_DAY2"])

In [75]:
cta_df.head(2)

,SERVICE_DATE,BUS_RIDES,RAIL_BOARDINGS,TOTAL_RIDES,YEAR,MONTH,WEEK_DAY,HOLIDAY
0,2001-01-01,297192,126455,423647,2001,1,Monday,Yes
1,2001-01-02,780827,501952,1282779,2001,1,Tuesday,No


In [76]:
#-------------- Check datatypes of each field and evaluate need for conversion

cta_df.dtypes

#-------------- Dataset is relatively small size for conversion object/stirng to category fields when 
# applicable to improve memory usage and enhance runtime performace

cta_df["WEEK_DAY"] = cta_df["WEEK_DAY"].astype("category")
cta_df["HOLIDAY"]  = cta_df["HOLIDAY"].astype("bool")
cta_df.dtypes

SERVICE_DATE      datetime64[ns]
BUS_RIDES                  int64
RAIL_BOARDINGS             int64
TOTAL_RIDES                int64
YEAR                       int64
MONTH                      int64
WEEK_DAY                category
HOLIDAY                     bool
dtype: object

In [77]:
#-------------- We are interested in analyzing crash datset since 2018, let see how far cta_df goes back

cta_df["YEAR"].value_counts()

2011    396
2014    396
2004    366
2008    366
2012    366
2016    366
2020    366
2013    365
2001    365
2005    365
2009    365
2019    365
2017    365
2015    365
2002    365
2006    365
2010    365
2018    365
2003    365
2007    365
2021    212
Name: YEAR, dtype: int64

In [78]:
#-------------- Get data since 2018 only
cta_df = cta_df[cta_df["YEAR"] >= 2018]
cta_df["YEAR"].value_counts()

2020    366
2019    365
2018    365
2021    212
Name: YEAR, dtype: int64

In [79]:
#-------------- Store final result as a csv file
cta_df.to_csv("chicago_data/chicago_cta_v2.csv", encoding="utf-8")